In [ ]:
import os

import numpy as np
import matplotlib.pyplot as plt

import popsynth
from popsynth.aux_samplers.trunc_normal_aux_sampler import TruncatedNormalAuxSampler
from popsynth.aux_samplers.lognormal_aux_sampler import LogNormalAuxSampler
from popsynth.aux_samplers.delta_aux_sampler import DeltaAuxSampler

from ..zusammen.synthetic_populations.aux_samplers import DurationSampler, LumSampler, EpeakObsSampler

import cosmogrb

In [ ]:
folder = "simulation"
if not os.path.exists(folder):
    os.mkdir(folder)

population_name = "corr_cpl_population"
universe_name = "corr_cpl_universe"

### Simulate a population in _popsynth_

In [ ]:
# pop_gen = popsynth.populations.SFRPopulation(r0=1/2, rise=1, decay=4, peak=1.5, r_max=7, a=0.1)
pop_gen = popsynth.populations.SFRPopulation(r0=1.5, rise=1, decay=4, peak=0.7, r_max=7, a=0.1)

t90 = LogNormalAuxSampler(name="t90", observed=False)
t90.mu=np.log10(10)
t90.tau=0.25

ep = LogNormalAuxSampler(name="ep", observed=False)
ep.mu= np.log(300)
ep.tau=0.4

alpha = TruncatedNormalAuxSampler(name="alpha", observed=False)
alpha.lower=-1.5
alpha.upper=0.0
alpha.mu=-1
alpha.tau=0.25

ep_tau = TruncatedNormalAuxSampler(name="ep_tau", observed=False)
ep_tau.lower=-2
ep_tau.upper=-1
ep_tau.mu=-1.5
ep_tau.tau=0.25

# nrest = LogNormalAuxSampler(name="nrest", observed=False)
# nrest.mu = np.log(1e52)
# nrest.tau = 0.1
nrest = DeltaAuxSampler(name="nrest", observed=False)
nrest.xp = 1e52

# gamma = TruncatedNormalAuxSampler(name="gamma", observed=False)
# gamma.mu = 1.5
# gamma.tau = 0.5
# gamma.lower = 1.0
# gamma.upper = 2.0
gamma = DeltaAuxSampler(name="gamma", observed=False)
gamma.xp = 1.5

duration = DurationSampler()
obs_lum = LumSampler()
ep_obs = EpeakObsSampler()

duration.set_secondary_sampler(t90)
obs_lum.set_secondary_sampler(ep)
obs_lum.set_secondary_sampler(nrest)
obs_lum.set_secondary_sampler(gamma)
ep_obs.set_secondary_sampler(ep)

pop_gen.add_observed_quantity(duration)
pop_gen.add_observed_quantity(obs_lum)
pop_gen.add_observed_quantity(ep_obs)
pop_gen.add_observed_quantity(alpha)
pop_gen.add_observed_quantity(ep_tau)


In [ ]:
population = pop_gen.draw_survey()
population.writeto(folder + population_name + ".h5")
population

### Simulate universe

In [ ]:
from ..examples.corr_cpl.corr_cpl_universe import GBM_CORR_CPL_Universe
universe = GBM_CORR_CPL_Universe('data/single_grb.h5', save_path="data")

universe.go(client=None)

universe.save(folder + universe_name + ".h5")

### View survey

In [ ]:
from cosmogrb.universe.survey import Survey

survey = Survey.from_file(folder + universe_name + ".h5")
survey.info()

In [ ]:
for i in range(len(survey)):
    print(f"GRB {i}")
    for j in ["alpha", "gamma", "Nrest"]:
        print(j, "=", survey['SynthGRB_' + str(i)].grb._source_params[j])
    print("z =", survey['SynthGRB_' + str(i)].grb.z)
    print("duration =", survey['SynthGRB_' + str(i)].grb.duration)
    print()

Light curves

In [ ]:
for grb_name in survey:
    fig, axes = plt.subplots(4,4,sharex=True,sharey=False,figsize=(10,10))
    row = 0
    col = 0
    for k,v in survey[grb_name].grb.items():
        ax = axes[row][col]

        lightcurve = v['lightcurve']

        lightcurve.display_lightcurve(dt=.5, ax=ax,lw=1,color='#25C68C')
        lightcurve.display_source(dt=.5,ax=ax,lw=1,color="#A363DE")
        lightcurve.display_background(dt=.5,ax=ax,lw=1, color="#2C342E")
        ax.set_xlim(-10, 30)
        ax.set_title(k,size=8)



        if col < 3:
            col+=1
        else:
            row+=1
            col=0

    #axes[3,2].set_visible(False)
    #axes[3,3].set_visible(False)